# SO(3) SCNN-based Patch Embedding

In this demo, we create the group equivariant steerable patchembedding block step-by-step, and test the equivariance of the generated embeddings. 

First we import the necessary modules : 

In [22]:
import escnn  # Library for efficient implementation of steerable group convolutional networks. 
import monai  # Established open-source medical imaging library.
import torch 
import numpy as np

First we will take a look at the original Vision transformer and the patchembedding block that is employed in the Vision Transformer image encoder of the SegVol architecture. The Vision Transformer that is used, is a direct implementation of the Vision Transformer from the paper "An image is worth 16x16 words: Transformers for Image Recognition At Scale <https://arxiv.org/abs/2010.11929>", but employs 3D-convolutions on 3-dimensional blocks instead of 2-dimensional patches to create embeddings of blocks within the volume instead of embeddings of patches within an image. 

The source code from the Monai ViT used is the following : 

```python

    class ViT(nn.Module):
    """
    Vision Transformer (ViT), based on: "Dosovitskiy et al.,
    An Image is Worth 16x16 Words: Transformers for Image Recognition at Scale <https://arxiv.org/abs/2010.11929>"

    ViT supports Torchscript but only works for Pytorch after 1.8.
    """

    def __init__(
        self,
        in_channels: int,
        img_size: Union[Sequence[int], int],
        patch_size: Union[Sequence[int], int],
        hidden_size: int = 768,
        mlp_dim: int = 3072,
        num_layers: int = 12,
        num_heads: int = 12,
        pos_embed: str = "conv",
        classification: bool = False,
        num_classes: int = 2,
        dropout_rate: float = 0.0,
        spatial_dims: int = 3,
        post_activation="Tanh",
    ) -> None:

        super().__init__()

        if not (0 <= dropout_rate <= 1):
            raise ValueError("dropout_rate should be between 0 and 1.")

        if hidden_size % num_heads != 0:
            raise ValueError("hidden_size should be divisible by num_heads.")

        self.classification = classification
        self.patch_embedding = PatchEmbeddingBlock(  
            in_channels=in_channels,
            img_size=img_size,
            patch_size=patch_size,
            hidden_size=hidden_size,
            num_heads=num_heads,
            pos_embed=pos_embed,
            dropout_rate=dropout_rate,
            spatial_dims=spatial_dims,
        )
        self.blocks = nn.ModuleList(
            [TransformerBlock(hidden_size, mlp_dim, num_heads, dropout_rate) for i in range(num_layers)]
        )
        self.norm = nn.LayerNorm(hidden_size)
        if self.classification:
            self.cls_token = nn.Parameter(torch.zeros(1, 1, hidden_size))
            if post_activation == "Tanh":
                self.classification_head = nn.Sequential(nn.Linear(hidden_size, num_classes), nn.Tanh())
            else:
                self.classification_head = nn.Linear(hidden_size, num_classes)  # type: ignore

    def forward(self, x):
        x = self.patch_embedding(x)
        if hasattr(self, "cls_token"):
            cls_token = self.cls_token.expand(x.shape[0], -1, -1)
            x = torch.cat((cls_token, x), dim=1)
        hidden_states_out = []
        for blk in self.blocks:
            x = blk(x)
            hidden_states_out.append(x)
        x = self.norm(x)
        if hasattr(self, "classification_head"):
            x = self.classification_head(x[:, 0])
        return x, hidden_states_out
```

When we analyze the code, we can see that the vision transformer starts with embedding the blocks within the volume, and passes this sequence of embeddings to a module list of transformer blocks that combine the embeddings within this sequence to a final image embedding. 

Let's inspect the source code of the PatchEmbeddingBlock to see in what way we are able to incorporate group equivariant features within this Vision Transformer. 

```python
    class PatchEmbeddingBlock(nn.Module):
    """
    A patch embedding block, based on: "Dosovitskiy et al.,
    An Image is Worth 16x16 Words: Transformers for Image Recognition at Scale <https://arxiv.org/abs/2010.11929>"

    Example::

        >>> from monai.networks.blocks import PatchEmbeddingBlock
        >>> PatchEmbeddingBlock(in_channels=4, img_size=32, patch_size=8, hidden_size=32, num_heads=4, pos_embed="conv")

    """

    def __init__(
        self,
        in_channels: int,
        img_size: Union[Sequence[int], int],
        patch_size: Union[Sequence[int], int],
        hidden_size: int,
        num_heads: int,
        pos_embed: str,
        dropout_rate: float = 0.0,
        spatial_dims: int = 3,
    ) -> None:
    
        super().__init__()

        if not (0 <= dropout_rate <= 1):
            raise ValueError("dropout_rate should be between 0 and 1.")

        if hidden_size % num_heads != 0:
            raise ValueError("hidden size should be divisible by num_heads.")

        self.pos_embed = look_up_option(pos_embed, SUPPORTED_EMBEDDING_TYPES)

        img_size = ensure_tuple_rep(img_size, spatial_dims)
        patch_size = ensure_tuple_rep(patch_size, spatial_dims)
        for m, p in zip(img_size, patch_size):
            if m < p:
                raise ValueError("patch_size should be smaller than img_size.")
            if self.pos_embed == "perceptron" and m % p != 0:
                raise ValueError("patch_size should be divisible by img_size for perceptron.")
        self.n_patches = np.prod([im_d // p_d for im_d, p_d in zip(img_size, patch_size)])
        self.patch_dim = int(in_channels * np.prod(patch_size))

        self.patch_embeddings: nn.Module
        if self.pos_embed == "conv":
            self.patch_embeddings = Conv[Conv.CONV, spatial_dims](
                in_channels=in_channels, out_channels=hidden_size, kernel_size=patch_size, stride=patch_size
            )
        elif self.pos_embed == "perceptron":
            # for 3d: "b c (h p1) (w p2) (d p3)-> b (h w d) (p1 p2 p3 c)"
            chars = (("h", "p1"), ("w", "p2"), ("d", "p3"))[:spatial_dims]
            from_chars = "b c " + " ".join(f"({k} {v})" for k, v in chars)
            to_chars = f"b ({' '.join([c[0] for c in chars])}) ({' '.join([c[1] for c in chars])} c)"
            axes_len = {f"p{i+1}": p for i, p in enumerate(patch_size)}
            self.patch_embeddings = nn.Sequential(
                Rearrange(f"{from_chars} -> {to_chars}", **axes_len), nn.Linear(self.patch_dim, hidden_size)
            )
        self.position_embeddings = nn.Parameter(torch.zeros(1, self.n_patches, hidden_size))
        self.dropout = nn.Dropout(dropout_rate)
        trunc_normal_(self.position_embeddings, mean=0.0, std=0.02, a=-2.0, b=2.0)
        self.apply(self._init_weights)

    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            trunc_normal_(m.weight, mean=0.0, std=0.02, a=-2.0, b=2.0)
            if isinstance(m, nn.Linear) and m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.LayerNorm):
            nn.init.constant_(m.bias, 0)
            nn.init.constant_(m.weight, 1.0)

    def forward(self, x):
        x = self.patch_embeddings(x)
        if self.pos_embed == "conv":
            x = x.flatten(2).transpose(-1, -2)
        embeddings = x + self.position_embeddings
        embeddings = self.dropout(embeddings)
        return embeddings
    
```

After closer inspection of `PatchEmbeddingBlock` class, it becomes clear that there are multiple ways to embed the patches/blocks within an intput. One of the options is to create a patch-embedding based on a convolution operation, that traverses over an image with a `kernel_size` $ k $ and `stride` $ s $ of the desired patch/block size. This convolutional operator maps a patch/block of `kernel_size` $ k $, to an embedding of `hidden_size` $ h $ defined by the number of output channels. This operation results in a new 3-Dimensional volume, with shape $\frac{{input\_image.shape[dim]}}{{kernel_size[dim]}}$ along all three dimensions and `hidden_size` $ h $ channels. This volume of patch embeddings is flattened along the spatial dimensions, resulting in a sequence of embeddings that can be passed to the transformer.



When we have a 3-dimensional medical image, and we have a rotated volume, the semantics of the volume's contents do not change as organs remain the same organs under different global rotations. This implicitly means that in an ideal learned representation we also expect a block to get the same (rotated) representation in such a scenario as it would have when it is not rotated. This means that if we would perform the inverse of the global rotation $R^{-1}$ on the new 3-dimensional volume with embeddings in each position $[x, y, z]$, we would like to see a representation that is the same as the volume of embeddings we would have gotten if we would not have rotated the volume in the first place. I.e., if we calculate a volume of block embeddings, we want that to be exactly the same as when we rotate the input volume, subsequently embed it and finally rotate it back to its original orientation. This concept is known as equivariance to 3-D rotation of the volume.


Let's test the equivariance to rotations of the 3D convolution that is applied in the original patch embedding block. Under the hood, this patch embedding block calls the torch.nn.functional.conv3d to get the volume of embeddings. This volume of embeddings is subsequently flattened along the spatial dimensions. So to test the equivariance of this block, we can test the equivariance of the conv3d operation from pytorch : 

In [23]:
# Load example image : 
from torch.nn import Conv3d
torch.manual_seed(42)

# Create a sample gray scale image with 3 spatial dimensions (first dim is the batch dim) :
image = torch.randn(1, 1, 128, 128, 128)

embedder = Conv3d(
    in_channels=image.shape[1],
    out_channels=768,   # hidden size of the embeddings
    kernel_size=(16, 16, 16),
    stride=16
)
embedding = embedder(image)
print(f'Shape of the embedded volume {embedding.squeeze().shape}')

# Rotate the volume by 90 degrees in the ZY plane (i.e. around the X axis)
image_x90 = image.rot90(k=1, dims=(2, 3))

embedding_x90 = embedder(image_x90)
print(f'Shape of the second embedded volume {embedding_x90.squeeze().shape}')

# Rotate the image 270 degrees further along the same dimension, and check whether the representations are equal
embedding_to_original_rotation = embedding_x90.rot90(k=3, dims=(2,3))

print(f'The two embeddings are equal : {torch.allclose(embedding_to_original_rotation, embedding)}')


Shape of the embedded volume torch.Size([768, 8, 8, 8])
Shape of the second embedded volume torch.Size([768, 8, 8, 8])
The two embeddings are equal : False


So the embeddings we have calculated are not equivariant to 3D rotations. 

Our proposed extension to this vision transformer applied within the SegVol network, is to make sure that these low-level features that are input to the vision transformer, are equivariant to rotations. We will achieve this by implementing a novel patch embedding block that embeds the blocks within a volume using a steerable convolution that is equivariant to the SO(3) group, which is the group of 3D rotations. 

# SO(3) equivariant features using `escnn` 
[https://github.com/QUVA-Lab/escnn](URL)

Group equivariant convolutional neural networks (GCNNs) impose additional inductive priors on classical convolutions based on symmetries. Imposed symmetries, such as rotational symmetry, reduce the total set of admissible representations that a network can possibly output. For instance, when we consider a traditional convolution operation on an image, a translation operation on the input image results in a corresponding translation in the output representation. The inductive bias, in this case, the translational symmetry, is incorporated through weight sharing and often results in more efficient representation learning because output features that violate the imposed symmetry are inherently excluded from the possibilities. The `escnn` package implements the modules that can be used to construct neural networks that are equivariant to a wider group of symmetries than just the translational symmetry seen in classical convolutions. The package provides implementations of modules that are equivariant under all isometries of the image plane $\mathbb{R}^2$ and of the 3D space $\mathbb{R}^3$, which means equivariant modules under the symmetries translations, rotations and reflections. 

GCNN's employ feature maps that are functions over the elements of the group. In the case of regular group-convolutions, this is achieved by computing a kernel transformed under (a discretely sampled amount of) the group elements, and storing the response for each of them. When dealing with the group of 3D rotations on medical imaging data, we would have to sample rotations along each of the three axes, rotate the 3D-kernel according to these samples, and constantly store all of the responses in memory. This quickly becomes a significant burden in terms of memory when we increase the granularity of rotations we consider. Conversely, steerable convolutional neural networks store the `fourier transform` of this feature map, resulting in a more efficient solution in terms of memory especially when considering infinite groups such as the group of rotations in 3D space. Consequently, we decided to implement a steerable convolution instead of a regular group convolution.  

The `escnn` package implements such equivariant convolutional modules through utilizing the feature maps of $E(n)$-equivariant steerable CNN's called feature fields. A feature field is defined as a space that is characterized by the way the field transforms under rotations and reflections. Such a field can be a scalar field (corresponding to a gray-scale image like in our case), or an n-dimensional vector field. These two types of features behave differently under rotations and reflections, as for example a rotated point remains the same point while a rotated vector results in a vector at a different global location and additionally a different orientation. When implementing modules in `escnn` equivariant to a certain isometry such as rotations in 3D space, a user has to specify what the input feature space is, and to what output feature space it should be mapped. An analogy can be drawn with traditional convolutions by considering that for a traditional convolution a pre-defined number of in-channels, is transformed to a pre-defined number of out-channels. Similar to traditional convolutions, in `escnn` a user can define a general convolution mapping on the 2D-image space by instantiating the module `R2conv` given the input, and output feature spaces, and similarly `R3conv` on the 3D-space. 

As briefly mentioned previously, we are dealing with 3-dimensional gray scale images (CT-scans), which corresponds to an `R3conv` module that has a scalar field as input field. 

Let's construct our 3D-Convolution equivariant to rotations in the 3D-space. 

We first have to specify the symmetry group action to which the transformation of the input feature field to the output feature field should be equivariant. We are looking for the group of rotations in the 3-dimensional space, which corresponds to the special orthogonal group on 3-dimensions $SO(3)$. We define this in `escnn` as follows:

```python

symmetry_group_action = escnn.gspaces.rot3don3d()
```

We now have defined the symmetry transformation on the 3D space, and now need to define how a feature vector field transforms under an action of the group. We do so by defining the input and output feature fields of the convolutional operator. In our case, we are dealing with grey 3D-images (CT-scans), which corresponds to a scalar field as input field. Scalar fields transform under the trivial representation of the $SO(3)$ group. The trivial representation is group representation that maps every element to 1. This means that a scalar feature (grey value) that is rotated, is equivariantly transformed by the identity mapping: 

In [24]:
symmetry_group_action = escnn.gspaces.rot3dOnR3()
in_field = escnn.nn.FieldType(symmetry_group_action, 1*[symmetry_group_action.trivial_repr]) # 1* because we are dealing with 1 input channel. 
in_field

[SO(3)_on_R3[so3]: {irrep_0 (x1)}(1)]

The last thing we now have to define for our equivariant patch embedding, is the output feature field. Similarly to the original patch embedding, we want the convolution to result in an embedding of `hidden_size` dimensionality. This can be achieved using the `escnn` library by defining the output feature field as being a `hidden_size` dimensional scalar field. 


In [25]:
out_field = escnn.nn.FieldType(symmetry_group_action, 768*[symmetry_group_action.trivial_repr]) # 1* because we are dealing with 1 input channel. 
out_field

[SO(3)_on_R3[so3]: {irrep_0 (x768)}(768)]

Patch everything together to create the group equivariant convolution : 

In [26]:
patch_embedder = escnn.nn.R3Conv(in_type=in_field, out_type=out_field, kernel_size=16, stride=16)
escnn.nn.init.generalized_he_init(patch_embedder.weights.data, patch_embedder.basisexpansion, cache=True)
for param in patch_embedder.parameters():
    print(f'count : {param.size()}')

count : torch.Size([768])
count : torch.Size([5376])


Let's test our new convolution operation on equivariance : 

In [27]:
def check_equivariance(patch_embedder, in_field):
    with torch.no_grad():
        x = torch.randn(1, 1, 128, 128 , 128)
        
        # the volumes rotated by 90 degrees in the ZY plane (i.e. around the X axis)
        x_x90 = x.rot90(1, (2, 3))
        x_x90 = escnn.nn.GeometricTensor(x_x90, in_field)
        yx = patch_embedder(x_x90).tensor

        # the volumes rotated by 90 degrees in the XZ plane (i.e. around the Y axis)
        x_y90 = x.rot90(1, (2, 4))
        x_y90 = escnn.nn.GeometricTensor(x_y90, in_field)
        yy = patch_embedder(x_y90).tensor

        # the volumes rotated by 90 degrees in the YX plane (i.e. around the Z axis)
        x_z90 = x.rot90(1, (3, 4))
        x_z90 = escnn.nn.GeometricTensor(x_z90, in_field)
        yz = patch_embedder(x_z90).tensor

        # the volumes rotated by 180 degrees in the XZ plane (i.e. around the Y axis)
        x_y180 = x.rot90(2, (2, 4))
        x_y180 = escnn.nn.GeometricTensor(x_y180, in_field)
        yy180 = patch_embedder(x_y180).tensor

        x = escnn.nn.GeometricTensor(x, in_field)
        y = patch_embedder(x).tensor

        # Rotate the outputs back to the original orientation, and check whether output matches with y.
        print('TESTING INVARIANCE:                     ')
        print('Equivariant to 90 degree rotations around X axis:  ' + ('True!' if torch.allclose(y, yx.rot90(3, (2,3)), atol=1e-3) else 'False'))
        print('Equivariant to 90 degree rotations around Y axis:  ' + ('True!' if torch.allclose(y, yy.rot90(3, (2,4)), atol=1e-3) else 'False'))
        print('Equivariant to 90 degree rotations around Z axis:  ' + ('True!' if torch.allclose(y, yz.rot90(3, (3,4)), atol=1e-3) else 'False'))
        print('Equivariant to 180 degree rotations around Y axis: ' + ('True!' if torch.allclose(y, yy180.rot90(2, (2,4)), atol=1e-4) else 'False'))

check_equivariance(patch_embedder, in_field)

TESTING INVARIANCE:                     
Equivariant to 90 degree rotations around X axis:  True!
Equivariant to 90 degree rotations around Y axis:  True!
Equivariant to 90 degree rotations around Z axis:  True!
Equivariant to 180 degree rotations around Y axis: True!


Let's also check gradient flow. 

In [28]:
initial_params = [x.clone() for x in patch_embedder.parameters()]
# Ten random images
X = torch.randn(10, 1, 128, 128 , 128)

# Sequence to go from embeddings to prediction
linear = torch.nn.Linear(768, 1)

optim = torch.optim.AdamW(patch_embedder.parameters())

# Ten random labels : 
y = torch.zeros((10, 1))

loss = torch.nn.BCELoss()

for i in range(X.shape[0]):
    item = X[i]
    item = escnn.nn.GeometricTensor(item.unsqueeze(0), in_field)

    item = patch_embedder(item).tensor
    # flatten spatial dimensions
    item = item.flatten(2).transpose(-1, -2)
    print(item.shape)
    # take average along spatial dimension
    item = torch.mean(item, dim=1)

    prediction = torch.nn.functional.sigmoid(linear(item))
    l = loss(prediction, y[i].unsqueeze(0))

    l.backward()
    optim.step()
    
after_params = [x.clone() for x in patch_embedder.parameters()]

# Parameters are correctly updated : 
for i, j in zip(initial_params, after_params):
    assert not torch.allclose(i, j)


torch.Size([1, 512, 768])
torch.Size([1, 512, 768])
torch.Size([1, 512, 768])
torch.Size([1, 512, 768])
torch.Size([1, 512, 768])
torch.Size([1, 512, 768])
torch.Size([1, 512, 768])
torch.Size([1, 512, 768])
torch.Size([1, 512, 768])
torch.Size([1, 512, 768])


This means that we now have a convolution operator that creates `hidden_size` dimensional features for each of the blocks in an image, equivariant to the $SO(3)$ group. 

This results in the following patch embedding block that incorporates rotational equivariant features at the lowest level : 

```python
    class SO3SteerablePatchEmbeddingBlock(nn.Module):
    def __init__(
        self,
        in_channels: int,
        img_size: Union[Sequence[int], int],
        # patch_size: Union[Sequence[int], int],
        patch_size: int,
        hidden_size: int,
        num_heads: int,
        dropout_rate: float = 0.0,
        spatial_dims: int = 3,
    ) -> None:
        """
        Args:
            in_channels: dimension of input channels.
            img_size: dimension of input image.
            patch_size: dimension of patch size.
            hidden_size: dimension of hidden layer.
            num_heads: number of attention heads.
            pos_embed: position embedding layer type.
            dropout_rate: faction of the input units to drop.
            spatial_dims: number of spatial dimensions.
        """

        super().__init__()

        if not (0 <= dropout_rate <= 1):
            raise ValueError("dropout_rate should be between 0 and 1.")

        if hidden_size % num_heads != 0:
            raise ValueError("hidden size should be divisible by num_heads.")

        img_size = ensure_tuple_rep(img_size, spatial_dims)
        patch_size = ensure_tuple_rep(patch_size, spatial_dims)
        for m, p in zip(img_size, patch_size):
            if m < p:
                raise ValueError("patch_size should be smaller than img_size.")
        self.n_patches = np.prod([im_d // p_d for im_d, p_d in zip(img_size, patch_size)])
        self.patch_dim = int(in_channels * np.prod(patch_size))

        # The model will be equivariant under planer rotations
        self.r3_act = escnn.gspaces.rot3dOnR3()

        # The group SO(3)
        self.Group = self.r3_act.fibergroup

        # The input field is a scalar field, because we are dealing with 3D-gray images (CT scans)
        in_type = enn.FieldType(self.r3_act, [self.r3_act.trivial_repr])
        self.input_type = in_type
        # The output is still a scalar, but we use #hidden_size channels to create the embeddings for the ViT
        out_type = enn.FieldType(self.r3_act, hidden_size*[self.r3_act.trivial_repr])
        
        # The 3D group equivariant convolution, that performs one pass on each block (similar to normal ViT)
        self.patch_embeddings = enn.R3Conv(in_type, out_type, kernel_size=8, stride=8)
        init.generalized_he_init(self.patch_embeddings.weights.data, self.patch_embeddings.basisexpansion, cache=True)

        grid_size = []
        for in_size, pa_size in zip(img_size, patch_size):
            grid_size.append(in_size // pa_size)
            self.position_embeddings = build_sincos_position_embedding(grid_size, hidden_size, spatial_dims)

    def forward(self, x):
        x = enn.GeometricTensor(x, self.input_type)
        x = self.patch_embeddings(x).tensor
        
        # Flatten all spatial dimensions into one, and add the learnable encoding
        x = x.flatten(2).transpose(-1, -2)
        embeddings = x + self.position_embeddings
        return embeddings
```